<h1>
    Libraries
</h1>

In [1]:
# these are all libraies used in the code
import pandas as pd 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import os
import getpass
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List ,Dict


C:\Users\yatin\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


<h1>
    Load Environment
</h1>

In [2]:
load_dotenv()
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(os.getenv("GOOGLE_API_KEY"))

<H1>
    Data Load
</H1>

In [3]:
Data = pd.read_excel("Copy of DT Data Champion Assignment.xlsx",sheet_name=["DataSet2 Advanced Screening","DataSet1 Basic Screening","Data Set4 Key Advanced2","DataSet3 Key Basic"])

<h3>
    Questions
</h3>

In [4]:
All_Basic_Question = Data["DataSet1 Basic Screening"].columns[2:7]

In [5]:
All_Advanced_Question = Data["DataSet2 Advanced Screening"].columns[2:12]

<h3>
    Data Set
</h3>

In [6]:
Advanced_Data = Data["DataSet2 Advanced Screening"]

In [7]:
Basic_Data = Data["DataSet1 Basic Screening"]

<h3>
    Key
</h3>

In [8]:
Basic_Key =  Data["DataSet3 Key Basic"]

In [9]:
Advanced_Key  = Data["Data Set4 Key Advanced2"]

<H1>
    Data Pre-Processing
</H1>

<h3>
    Basic Data Pre-Processing
</h3>

In [10]:
Basic_Data.columns = [Basic_Data.columns[0], Basic_Data.columns[1], "Q1","Q2","Q3","Q4","Q5",
                      Basic_Data.columns[7],Basic_Data.columns[8],
                      Basic_Data.columns[9],"Ans1","Ans2","Ans3","Ans4","Ans5"]

In [11]:
Basic_Key.columns = ["1","2","Question", "Marks","Option"]

In [12]:
Basic_Key =  Basic_Key.drop(["1","2"],axis = 1)

In [13]:
Basic_Key  =  Basic_Key.dropna(how='all', axis=0 )

In [14]:
Question = None
Basic_Key_map = {}
count= 0 
for idx, i in Basic_Key.iterrows():
    if pd.notna(i.Question):
        Question  = All_Basic_Question[count]
        Basic_Key_map[Question] = []
        count+=1
    Basic_Key_map[Question].append(i.Option)

<h3>
    Advanced Data Pre-Processing
</h3>

In [15]:
Advanced_Data.columns = [Advanced_Data.columns[0], Advanced_Data.columns[1], "Q1","Q2","Q3","Q4","Q5",
                         "Q6","Q7","Q8","Q9","Q10",
                         "Name",
                         Advanced_Data.columns[13],Basic_Data.columns[14],"Ans1","Ans2","Ans3","Ans4",
                         "Ans5","Ans6","Ans7","Ans8","Ans9","Ans10"]

In [16]:
Advanced_Key.columns = ["1","2","Question", "Marks","Option"]

In [17]:
Advanced_Key =  Advanced_Key.drop(["1","2"],axis = 1)

In [18]:
Advanced_Key  =  Advanced_Key.dropna(how='all', axis=0 )

In [19]:
Question = None
count = 0 
Advanced_Key_map = {}
for idx, i in Advanced_Key.iterrows():
    if pd.notna(i.Question):
        Question  =  All_Advanced_Question[count]
        Advanced_Key_map[Question] = []
        count+=1
    Advanced_Key_map[Question].append(i.Option)

In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [21]:
Query = """ You are an Human analyst trying to evaluate candiate based on Risk-taking(adventurousness), self-learning, 
            sense of achievement , cross-functional thinking and ability to take feedback. For this purpose few question are prepared and 
            there are 4 options from which user can select any one of them. You have to rate out of 4 every option. You must give rating for each
            and every 5 criteria( Risk-taking(adventurousness), self-learning, sense of achievement , cross-functional thinking and ability to take feedback)
            mentioned above. Also while rating option you should not biased to any option weight every option indvidually. 
            You have to only give option number criteria you are rating and rating for the criteria nothing else. 
            It sructure must be like this [   Risk-taking(adventurousness):3 , self-learning:2, sense of achievement , 
            cross-functional thinking:1 ability to take feedback:5]. {format_instructions}. {question} and these are {options} to the question.
            """

In [22]:
class Output(BaseModel):
    Question: str = Field(description="Question number in the form Q1 only nothing else. ")
    Options:List[Dict[str,str]] = Field(description="Specify all options")
    Rating: Dict[int,Dict[str,str]] = Field(description="Rating on the basis of given criterias of options based on question. Same format as input option, do not use keyword option to depect option use interger 0,1,2,3")

In [23]:
def Rating(Query, Dict, LLM):
    parser = JsonOutputParser(pydantic_object=Output)
    prompt = PromptTemplate(
            template=Query,
            input_variables=["question", "options"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
    chain = prompt | LLM | parser
    result = []
    for i in Dict.keys():
        result.append(chain.invoke({
            "question":i,
            "options":Dict[i]
            }))
    return result

In [24]:
basic_ratings = Rating(Query, Basic_Key_map, llm)

In [26]:
advanced_ratings = Rating(Query,Advanced_Key_map,llm)

In [27]:
Basic_Key = Basic_Key.ffill()
Advanced_Key = Advanced_Key.ffill()

In [28]:
for i in Basic_Key['Question'].unique():
    q1 = Basic_Key.loc[Basic_Key['Question']==i]['Option'].unique()
    count = 0 
    for j in q1:
        Basic_Key.loc[(Basic_Key['Question'] == i) & (Basic_Key['Option'] == j), 'Option']= count
        Basic_Data.loc[Basic_Data[i] == j,i] = count
        count+=1

In [29]:
 for i in Advanced_Key['Question'].unique():
    q1 = Advanced_Key.loc[Advanced_Key['Question']==i]['Option'].unique()
    count = 0 
    for j in q1:
        Advanced_Key.loc[(Advanced_Key['Question'] == i) & (Advanced_Key['Option'] == j), 'Option']= count
        Advanced_Data.loc[Advanced_Data[i] == j,i] = count
        count+=1

In [30]:
Basic_Data['self-learning'] = 0
Basic_Data['sense of achievement'] = 0
Basic_Data['cross-functional thinking'] = 0
Basic_Data['ability to take feedback'] = 0
Basic_Data['Risk-taking(adventurousness)'] = 0


Advanced_Data['self-learning'] = 0
Advanced_Data['sense of achievement'] = 0
Advanced_Data['cross-functional thinking'] = 0
Advanced_Data['ability to take feedback'] = 0
Advanced_Data['Risk-taking(adventurousness)'] = 0

In [31]:
para = ['self-learning','sense of achievement','cross-functional thinking','ability to take feedback','Risk-taking(adventurousness)']
Basic_question = ['Q1', 'Q2','Q3','Q4','Q5']
Advanced_question = ['Q1', 'Q2','Q3','Q4','Q5','Q6', 'Q7','Q8','Q9','Q10']

In [32]:
for itr, i in Basic_Data.iterrows():
    for j in range(0,len(Basic_question)):
        for k in para:
            Basic_Data.at[itr,k] = Basic_Data.at[itr,k] + int(basic_ratings[j]['Rating'][str(i[Basic_question[j]])][k])

In [33]:
for itr, i in Advanced_Data.iterrows():
    for j in range(0,len(Advanced_question)):
        for k in para:
            t = 0 
            try:
                 t =  int(advanced_ratings[j]['Rating'][str(i[Advanced_question[j]])][k])
            except:
                 t = 0
            Advanced_Data.at[itr,k] = Advanced_Data.at[itr,k] + t

In [43]:
Suggestive_query = """you are an expert reviewer who has to give feedback to students based on their response 
in assessment. There were two test conducte one was basic while other was advanced. All students had given basic test but 
some had not attempted advanced test. There 5 criterion(Risk-taking(adventurousness), self-learning, sense of achievement , cross-functional thinking and ability to 
take feedback) to evaluate each option of a question. Each option given score out of 4 based on criteria. Since it is not possible
to achive full 4 points in criteria. However best option usually achive full marks in 2-3 criteria. Since all option marked by students are marked and evaluated 
score of eachh criterion is added to get final total score of each criterion. Score basic and adanced test are collectively. Since 
none student can get full score in criterion reciving score above 60 in a specific criterion is considered good for that criterion. 
write review recommending students area where they can work and improve.Also give some suggestion student can follow to improve above criterion.  
Basic Test Scores based on 5 criterion:
self-learning:{Basic-self-learning},
sense of achievement:{Basic-sense of achievement},
cross-functional thinking:{Basic-cross-functional thinking},
ability to take feedback:{Basic-ability to take feedback} and
Risk-taking(adventurousness): {Basic-Risk-taking(adventurousness)}

Advanced Test Scores based on 5 criterion:
self-learning:{Advanced-self-learning},
sense of achievement:{Advanced-sense of achievement},
cross-functional thinking:{Advanced-cross-functional thinking},
ability to take feedback:{Advanced-ability to take feedback} and
Risk-taking(adventurousness): {Advanced-Risk-taking(adventurousness)}
{format_instructions}
"""

In [39]:
Merge = pd.merge(Basic_Data,Advanced_Data, on="Name", how="outer")

In [50]:
class Review_Structure(BaseModel):
    feedback: str = Field(description="Give feedback based on scores of Basic and advanced criterion")
    suggestion: str = Field(description="Give suggestion to improve score ")
  

In [51]:
def Rating(Row, LLM):
    parser = JsonOutputParser(pydantic_object=Review_Structure)
    prompt = PromptTemplate(
            template=Suggestive_query,
            input_variables=["Basic-self-learning", "Basic-sense of achievement","Basic-cross-functional thinking", "Basic-ability to take feedback"
                            ,"Basic-Risk-taking(adventurousness)","Advanced-self-learning","Advanced-sense of achievement","Advanced-cross-functional thinking",
                             "Advanced-ability to take feedback","Advanced-Risk-taking(adventurousness)"
                            ],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
    chain = prompt | LLM | parser
    result = chain.invoke({
            "Basic-self-learning":Row['self-learning_x'], "Basic-sense of achievement":Row['sense of achievement_x'],"Basic-cross-functional thinking":Row['cross-functional thinking_x'], "Basic-ability to take feedback":Row['ability to take feedback_x']
            ,"Basic-Risk-taking(adventurousness)":Row['Risk-taking(adventurousness)_x'],"Advanced-self-learning":Row['self-learning_y'],"Advanced-sense of achievement":Row['sense of achievement_y'],"Advanced-cross-functional thinking":Row['cross-functional thinking_y'],
            "Advanced-ability to take feedback":Row['ability to take feedback_y'],"Advanced-Risk-taking(adventurousness)":Row['Risk-taking(adventurousness)_y']
            })
    return result

In [85]:
feedback = pd.DataFrame(columns=["Name","Feedback","Suggestion"])   

In [89]:
for itr , row in Merge.iterrows():
    response = Rating(row,llm)
    feedback = feedback._append({"Name":row['Name'],"Feedback":response["feedback"],"Suggestion":response["suggestion"]},ignore_index=True)

In [93]:
feedback.to_excel("Feedback.xlsx",index=False)